### Neural Network Creation and Training:

The following code creates a Neural Network from scratch, and trains the Neural Network based on the MNIST dataset

In [6]:
import torch
import torch.nn as nn
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

import random
import numpy as np

from helpers import *

In [7]:
device = torch.device('cuda') if torch.cuda.is_available() else "cpu"

In [8]:
# Structure adopted partially from the PyTorch Documentation page, basic Neural Network construction:

# Note: The input images from MNIST will be in 28x28 dimensions, so linear input should be in 784:

# Create the architecture for a very basic Neural Network:
class MNISTNeuralNetwork(nn.Module):
    def __init__(self, input_size, output_size):
      super().__init__()
      self.flatten = nn.Flatten()
      self.linear_relu_stack = nn.Sequential(
          nn.Linear(input_size, 256),
          nn.ReLU(),
          nn.Linear(256, 256),
          nn.ReLU(),
          nn.Linear(256, output_size),
      )

    def forward(self, x):
      x = self.flatten(x)
      logits = self.linear_relu_stack(x)
      return logits

### Loading the MNIST Data:

In order to load the MNIST Data into the notebook, we will need to make use of datasets and DataLoader modules as provided in PyTorch:

In [9]:
training_dataset = datasets.MNIST(root='data/', train=True, download=True, transform=transforms.ToTensor())
training_loader = DataLoader(training_dataset, batch_size=64)

test_dataset = datasets.MNIST(root='data/', train=False, download=True, transform=transforms.ToTensor())
test_loader = DataLoader(test_dataset, batch_size=64)

100%|██████████| 9.91M/9.91M [00:00<00:00, 17.9MB/s]
100%|██████████| 28.9k/28.9k [00:00<00:00, 481kB/s]
100%|██████████| 1.65M/1.65M [00:00<00:00, 3.87MB/s]
100%|██████████| 4.54k/4.54k [00:00<00:00, 6.61MB/s]


In [10]:
# Initialize the Network:
model = MNISTNeuralNetwork(input_size = 784, output_size=10).to(device)

# Declare Loss Function and Optimizer:
loss_fcn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.01)
optimizer.zero_grad()

### Training the Actual Network:

For this project we will train the model on CLEAN data, but will test the neural network on noisy data. For this reason, we will need to alter the test dataset images later on in the notebook. However, for now - we will train the neural network that we have created over 50 epochs.

In [12]:
epochs = 50
past_training_loss = np.inf

for epoch in range(epochs):

  training_loss = 0

  for idx, (data, label) in enumerate(training_loader):
    data = data.to(device)
    labels = label.to(device)

    data_flat = data.reshape(-1, 784)

    # Make the prediction based on the NN model:
    scores = model(data_flat)

    # Compute the loss value based on the pre-defined loss function:
    loss_val = loss_fcn(scores, labels)

    # Backpropagation, allow for:
    optimizer.zero_grad()
    loss_val.backward()
    optimizer.step()

    training_loss += loss_val.item() * data.size(0)

  print(f"Epoch Number: {epoch}, Training Loss: {training_loss}")
  epoch += 1

Epoch Number: 0, Training Loss: 101300.79993057251
Epoch Number: 1, Training Loss: 32304.3777384758
Epoch Number: 2, Training Loss: 23235.27440929413
Epoch Number: 3, Training Loss: 20239.67621064186
Epoch Number: 4, Training Loss: 18430.0731613636
Epoch Number: 5, Training Loss: 17063.020985126495
Epoch Number: 6, Training Loss: 15927.504470586777
Epoch Number: 7, Training Loss: 14932.963267803192
Epoch Number: 8, Training Loss: 14041.836070775986
Epoch Number: 9, Training Loss: 13231.25315272808
Epoch Number: 10, Training Loss: 12489.769403636456
Epoch Number: 11, Training Loss: 11809.711961269379
Epoch Number: 12, Training Loss: 11185.74990093708
Epoch Number: 13, Training Loss: 10611.091792285442
Epoch Number: 14, Training Loss: 10081.262268006802
Epoch Number: 15, Training Loss: 9593.343569934368
Epoch Number: 16, Training Loss: 9143.080515027046
Epoch Number: 17, Training Loss: 8724.285730421543
Epoch Number: 18, Training Loss: 8333.70962792635
Epoch Number: 19, Training Loss: 79

### Testing Accuracy and Loss:

Now we will test the model on the testing data, and assess the loss values and accuracy:

In [ ]:
def generate_test_loss(model, test_loader):

  """Determines the test dataset loss, given a model

  Parameters:
  - model (MNISTNeuralNetwork) - The Machine Learning model used on the test dataset
  - test_loader (torch.utils.data.dataloader.DataLoader) - The dataloader that is used to load the test data into the model

  Returns:
  - None (prints accuracy and test_loss metrics)"""

  total_correct = 0
  total_counter = 0

  test_loss = 0

  for data, labels in test_loader:

    data = data.to(device)
    labels = labels.to(device)

    test_data_flat = data.reshape(-1, 784)

    predictions = model(test_data_flat)

    for pred, label in zip(predictions, labels):
      if torch.argmax(pred).item() == label.item():
        total_correct += 1
      total_counter += 1

    loss = loss_fcn(predictions, labels)
    test_loss += loss.item() * data.size(0)

  print(f"Accuracy: {(total_correct/total_counter) * 100}")
  print(f"Test Loss: {test_loss/len(test_loader)}")

In [13]:
generate_test_loss(model, test_loader)

Accuracy: 97.50999999999999
Test Loss: 4.9732838382078395


In [ ]:
 # Save the model:
torch.save(model.state_dict(), 'trained_model.pth')

### Adding Noise to the Test Dataset:

One of the main purposes of this project is to see whether the Neural Network is robust enough to handle noise in a dataset. Given that the Neural Network is trained on a clean dataset, we want to see if adding noise from other distributions would make a huge difference in Neural Network classification accuracy.

To add noise to the images, we use a Gaussian Mixture Model (GMM), and experiment with other models to add anomalous perturbations to the images. First we need to create a custom transformation:

In [14]:
class AddRandomizedGaussianNoise(object):

  # Adapted from this thread: https://discuss.pytorch.org/t/how-to-add-noise-to-mnist-dataset-when-using-pytorch/59745

  def __init__(self, mean, st_dev):
    self.mean = mean
    self.st_dev = st_dev

  def __call__(self, tensor):
    return tensor + torch.randn(tensor.size()) * self.st_dev + self.mean

  def __repr__(self):
    return self.__class__.__name__ + f"(Mean: {self.mean}, Standard Deviation: {self.st_dev})"

In [15]:
# Redefine the Testing Dataset to include randomized noise:

test_dataset = datasets.MNIST(root='data/', train=False, download=True, transform=transforms.Compose([transforms.ToTensor(),
                                                                                                      AddRandomizedGaussianNoise(random.randint(0, 5), 1)]))
test_loader = DataLoader(test_dataset, batch_size=64)

In [16]:
generate_test_loss(model, test_loader)

Accuracy: 9.790000000000001
Test Loss: 5579.413746050209


While running this, you should probably see that the test accuracy here (with noise) is much less than the test accuracy on similar images. This was to be expected especially considering we're adding extremely randomized noise to the test dataset.

### SMT Solvers:

Satisfiable Modulo Theories (SMT) solvers are designed to assess whether logical formulas are satisfiable or not. In practical terms, it can be used to determine whether the design of a certain hardware/software is correct. Here we use SMT solvers to verify the "correctness" of the Neural Network.

In [ ]:
!pip install PySMT